# Dicionário dos Dados
| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

#Instalando o Spark

In [53]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()

# Inciando o Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [54]:
spark

#Leitura dos Dados

In [55]:
from pyspark.sql import functions as f
from pyspark.sql import types as t

In [56]:
data = spark.read.json("/content/drive/MyDrive/Colab Notebooks/ChallengeDataScience2°Ed/data/dataset_bruto.json")
data.show(5)

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



#Transformar cada Campo de "Anúncio" em uma Coluna Separada


In [57]:
anuncios = data.select("anuncio").withColumnRenamed("anuncio", "anuncios")
anuncios.show(5)

+--------------------+
|            anuncios|
+--------------------+
|{0, [], [16], [0]...|
|{0, [], [14], [0]...|
|{0, [1026], [1026...|
|{0, [120], [120],...|
|{0, [3], [3], [0]...|
+--------------------+
only showing top 5 rows



In [58]:
# Transformar cada campo da coluna "anuncio" em uma coluna separada
anuncios = anuncios.select(f.col("anuncios.*"))

In [59]:
anuncios.show(5, False)

+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                        |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|0    |[]    

In [60]:
anuncios.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

# Filtrar a base de dados

In [61]:
# Tabela de Frequência de tipo_uso
anuncios.groupBy("tipo_uso").count().show(truncate=False)

+-----------+-----+
|tipo_uso   |count|
+-----------+-----+
|Comercial  |4542 |
|Residencial|84541|
+-----------+-----+



In [62]:
# Tabela de Frequência de tipo_unidade
anuncios.groupBy("tipo_unidade").count().show(truncate=False)

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|Outros      |11963|
|Apartamento |66801|
|Casa        |10319|
+------------+-----+



In [63]:
# Tabela de Frequência de tipo_anuncio
anuncios.groupBy("tipo_anuncio").count().show(truncate=False)

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|Usado       |88827|
|Lançamento  |256  |
+------------+-----+



In [64]:
# Filtragem
anuncios\
  .filter(anuncios["tipo_uso"] == "Residencial")\
  .filter(anuncios["tipo_unidade"] == "Apartamento")\
  .filter(anuncios["tipo_anuncio"] == "Usado")\
  .show(5, False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |endereco                                                                                                               |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                    |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+--------------

# Transformar as colunas dos cômodos dos imóveis de listas para inteiros

In [65]:
# Retira os itens das Listas
anuncios = anuncios.withColumns(
  {
    "area_total": f.col("area_total").getItem(0),
    "area_util": f.col("area_util").getItem(0),
    "banheiros": f.col("banheiros").getItem(0),
    "caracteristicas": f.col("caracteristicas").getItem(0),
    "quartos": f.col("quartos").getItem(0),
    "suites": f.col("suites").getItem(0),
    "vaga": f.col("vaga").getItem(0),
    "valores": f.col("valores").getItem(0)
  }
)

In [66]:
# Muda os tipos das colunas Numéricas
anuncios = anuncios.withColumns(
  {
    "andar": anuncios["andar"].cast(t.IntegerType()),
    "banheiros": anuncios["andar"].cast(t.IntegerType()),
    "quartos": anuncios["quartos"].cast(t.IntegerType()),
    "suites": anuncios["suites"].cast(t.IntegerType()),
    "vaga": anuncios["vaga"].cast(t.IntegerType()),
    "area_total": anuncios["area_total"].cast(t.IntegerType()),
    "area_util": anuncios["area_util"].cast(t.IntegerType())
  }
)

In [67]:
anuncios.printSchema()

root
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: struct (nullable = true)
 |    |-- condominio: string (nullable = tru

# Tratamento de informações sobre localização

In [68]:
anuncios.show(5, False)

+-----+----------+---------+---------+-----------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+--------------------------+
|andar|area_total|area_util|banheiros|caracteristicas  |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                   |
+-----+----------+---------+---------+-----------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+--------------------------+
|0    |NULL      |16       |0        |NULL             |{Centro, 20061003, Rio de Jan

In [70]:
# Extrai os valores de "bairro" e "zona" da coluna "endereço" e removendo a coluna
anuncios = anuncios.join(
  other=anuncios.select("id", "endereco.bairro", "endereco.zona"),
  on="id",
  how="inner")\
.drop("endereco")

In [71]:
anuncios.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: struct (nullable = true)
 |    |-- condominio: string (nullable = true)
 |    |-- iptu: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |    |-- valor: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



# Transformar cada campo da coluna "valores" em uma coluna separada

In [77]:
# Transforma os campos da coluna "valores" em colunas e a remove
anuncios = anuncios.join(
  other=anuncios.select("id", "valores.condominio", "valores.iptu", "valores.tipo", "valores.valor"),
  on="id",
  how="inner")\
.drop("valores")

In [78]:
anuncios.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



# Tratamento para a coluna de valores

In [84]:
anuncios.groupBy("tipo").count().show()

+-------+-----+
|   tipo|count|
+-------+-----+
|Aluguel|  811|
|  Venda|88272|
+-------+-----+



In [90]:
anuncios = anuncios.filter(anuncios["tipo"] == "Venda")

In [92]:
anuncios.show(5, False)

+------------------------------------+-----+----------+---------+---------+------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+-----+-----+
|id                                  |andar|area_total|area_util|banheiros|caracteristicas   |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                 |zona        |condominio|iptu|tipo |valor|
+------------------------------------+-----+----------+---------+---------+------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+-----+-----+
|dde8b6b7-111e-4ad1-a32b-0964d18ce497|0    |10        |10       |0        |NULL              |NULL   |NULL  |Usado       |Outros      |Comercial  |1   |Centro                 |Zona Central|NULL      |NULL|Venda|20000|
|03a386b6-7ab8-4eff-891d-f8a16efc1989|0    |43        |43       |0        |Churrasqueira     |2      |NULL  |Usado       |Aparta

In [96]:
# Transforma as colunas de valor monetário no tipo Double
anuncios = anuncios.withColumns(
  {
    "condominio": f.col("condominio").cast(t.DoubleType()),
    "iptu": f.col("iptu").cast(t.DoubleType()),
    "valor": f.col("valor").cast(t.DoubleType())
  }
)

In [97]:
anuncios.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



# Salvar os dados no formato PARQUET e CSV


In [99]:
anuncios.write.parquet(path="/content/drive/MyDrive/Colab Notebooks/ChallengeDataScience2°Ed/data/parquet", mode="overwrite")

In [100]:
anuncios.write.csv(path="/content/drive/MyDrive/Colab Notebooks/ChallengeDataScience2°Ed/data/csv", mode="overwrite", header=True)

# Comparar o desempenho da leitura

In [102]:
%%time
parquet_read_data = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/ChallengeDataScience2°Ed/data/parquet")

CPU times: user 2.28 ms, sys: 27 µs, total: 2.31 ms
Wall time: 314 ms


In [103]:
%%time
csv_read_data = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/ChallengeDataScience2°Ed/data/csv", header=True)

CPU times: user 10 ms, sys: 462 µs, total: 10.5 ms
Wall time: 673 ms


In [105]:
parquet_read_data.show(5, False)

+------------------------------------+-----+----------+---------+---------+------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+------+-----+-------+
|id                                  |andar|area_total|area_util|banheiros|caracteristicas   |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                 |zona        |condominio|iptu  |tipo |valor  |
+------------------------------------+-----+----------+---------+---------+------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+------+-----+-------+
|dde8b6b7-111e-4ad1-a32b-0964d18ce497|0    |10        |10       |0        |NULL              |NULL   |NULL  |Usado       |Outros      |Comercial  |1   |Centro                 |Zona Central|NULL      |NULL  |Venda|20000.0|
|03a386b6-7ab8-4eff-891d-f8a16efc1989|0    |43        |43       |0        |Churrasqueira     |2      |NULL  |Usa

In [106]:
csv_read_data.show(5, False)

+------------------------------------+-----+----------+---------+---------+------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+------+-----+-------+
|id                                  |andar|area_total|area_util|banheiros|caracteristicas   |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                 |zona        |condominio|iptu  |tipo |valor  |
+------------------------------------+-----+----------+---------+---------+------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+------+-----+-------+
|dde8b6b7-111e-4ad1-a32b-0964d18ce497|0    |10        |10       |0        |NULL              |NULL   |NULL  |Usado       |Outros      |Comercial  |1   |Centro                 |Zona Central|NULL      |NULL  |Venda|20000.0|
|03a386b6-7ab8-4eff-891d-f8a16efc1989|0    |43        |43       |0        |Churrasqueira     |2      |NULL  |Usa

In [107]:
spark.stop()